In [ ]:
# Calculamos el TMY para todos los puntos.
import os

import itertools as it

import numpy as np
import pandas as pd

import xarray as xr

In [ ]:
def interpolate(ds):
    

In [ ]:
# Cargamos el archivo.
path_d = "../temp/radiacion/"
path_r = "../temp/TMY_years/"
path_y = "../temp/TMY_years/"

TZ = -6

files = os.listdir(path_d)
files.sort()
if ".DS_Store" in files: files.remove(".DS_Store")
for f in files:
    print(f"{f[-5:-3]}", end=",")
    df_y = pd.read_csv(path_y + f)
    with xr.open_dataset( path_d + f ) as ds:
        ds["time"] = ds.indexes["time"].shift(TZ, "h")
        # Quitamos los 29 de febrero.
        ds = ds.sel( time = ~( (ds.time.dt.month == 2)
            & (ds.time.dt.day == 29) ) )
        
        ds_tmy = ds.isel({"time": slice(0, 365)}).copy()
        ds_tmy["time"] = pd.date_range( "01/01/2001 00:00:00",
            "31/12/2001 23:00:00", freq = "H" )
        
        for lat in ds["latitude"]:
            for lon in ds["longitude"]:
                years = df_y.loc[(lat, lon)]
                ds_y = ds.sel({"time": years})
                ds_m = []
                for i in range(years.shape[0]):
                    ds_m.append( ds.where( ( (ds["time"].dt.month==(i+1)) &
                        (ds["time"].dt.year==years[i]) ), drop = True ) )
                    ds_tmy = xr.concat(ds_m, dim = "time")

In [ ]:
years = np.arange(1990, 2002)
TZ = -6
ds_temp = ds.convert_calendar("noleap")

ds_temp["time"] =  ds_temp.indexes["time"].shift(TZ, "h")

ds_m = []
for i in range(years.shape[0]):
    ds_m.append( ds.where( ( (ds["time"].dt.month==(i+1)) &
        (ds["time"].dt.year==years[i]) ), drop = True ) )
ds_tmy = xr.concat(ds_m, dim = "time")

In [ ]:
df_tmy = ds_tmy.to_dataframe()
time = df_tmy.index.get_level_values("time")
m = 1
front = df_tmy.loc[ ( ( ( time.month.isin([m]) & time.hour.isin(range(18, 24)) )
    | ( time.month.isin([1 + 1]) & time.hour.isin(range(0, 6)) ) ), lat, lon ), v ]